<a href="https://colab.research.google.com/github/andylolu2/simpleGEMM/blob/master/colab/simpleGEMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone --recurse-submodules https://github.com/andylolu2/simpleGEMM.git
%cd simpleGEMM

Cloning into 'simpleGEMM'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 41 (delta 21), reused 18 (delta 8), pack-reused 0
Receiving objects: 100% (41/41), 17.79 KiB | 2.96 MiB/s, done.
Resolving deltas: 100% (21/21), done.
Submodule 'src/cutlass' (https://github.com/NVIDIA/cutlass.git) registered for path 'cutlass'
Cloning into '/content/simpleGEMM/cutlass'...
remote: Enumerating objects: 26417, done.        
remote: Counting objects: 100% (7329/7329), done.        
remote: Compressing objects: 100% (984/984), done.        
remote: Total 26417 (delta 6693), reused 6358 (delta 6345), pack-reused 19088        
Receiving objects: 100% (26417/26417), 41.27 MiB | 21.09 MiB/s, done.
Resolving deltas: 100% (20146/20146), done.
Submodule path 'cutlass': checked out '19f3cc33f1642b490ed7126ea0141f79c0045527'
/content/simpleGEMM


In [2]:
!mkdir build
%cd build
!cmake ..
!make

/content/simpleGEMM/build
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- The CUDA compiler identification is NVIDIA 12.2.140
-- Detecting CUDA compiler ABI info
-- Detecting CUDA compiler ABI info - done
-- Check for working CUDA compiler: /usr/local/cuda/bin/nvcc - skipped
-- Detecting CUDA compile features
-- Detecting CUDA compile features - done
-- Configuring done (4.8s)
-- Generating done (0.0s)
-- Build files have been written to: /content/simpleGEMM/build
[ 16%] Building CUDA object CMakeFiles/main.dir/main.cu.o
[ 33%] Linkin

In [5]:
!echo "Ours:"
!./main 4096 4096 4096 1000
!echo "Reference:"
!./reference 4096 4096 4096 1000

Ours:
Time elapse: 3984.36ms
TFLOPS: 34.4946
Reference:
Time elapse: 3942.95ms
TFLOPS: 34.8568


In [6]:
# Compare against PyTorch (CuBLAS)
import torch

M, N, K = 4096, 4096, 4096
iters = 1000

A = torch.randn(M, K, dtype=torch.float16, device="cuda")
B = torch.randn(N, K, dtype=torch.float16, device="cuda").T

start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)
start.record()
for _ in range(iters):
    C = A @ B
end.record()
torch.cuda.synchronize()

total_duration = start.elapsed_time(end)  # in ms
tflops = 2 * M * N * K * iters / (total_duration / 1000) / 1e12
print(f"Time elapse: {total_duration:.2f}ms")
print(f"TFLOPS: {tflops:.4f}")

Time elapse: 6606.95ms
TFLOPS: 20.8022


> TODO: Why is PyTorch so slow?